In [1]:
'''
this script will generate tiles of raw images and corresponding labels
- tile size must be divisible by 32 for unet model
'''

'\nthis script will generate tiles of raw images and corresponding labels\n- tile size must be divisible by 32 for unet model\n'

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [3]:
## path to directories

raw_data_path = r'\\babyserverdw3\PW Cloud Exp Documents\Lab work documenting\W-22-09-10 AT Build Competent multi task DL model for tissue labeling\dataset\1101\raw'

save_path = r'\\babyserverdw3\PW Cloud Exp Documents\Lab work documenting\W-22-09-10 AT Build Competent multi task DL model for tissue labeling\dataset\1101\tiled'

In [4]:
def find_best_tile_size(im_dim0, im_dim1, min_tile_size=480, max_tile_size=1024):
    possible_sizes = np.arange(min_tile_size, max_tile_size, 32) # must be divisible by 32 for unet
    remainders = [] # number of pixels lost
    for i in possible_sizes:
        r0 = (im_dim0 % i) * im_dim1
        r1 = (im_dim1 % i) * im_dim0
        overlap = ((im_dim0 % i) * (im_dim1 % i))
        remainders.append(r0 + r1 - overlap)
    best_size = possible_sizes[np.argmax(remainders)]
    print('best tile size: {}, num pixels lost: {}'.format(best_size, np.max(remainders)))
    return best_size

In [5]:
tile_size = find_best_tile_size(2048, 2880)

tile_folder_name = '{}x{}'.format(tile_size, tile_size)

tile_save_path = os.path.join(save_path, tile_folder_name)

if not os.path.exists(tile_save_path):
    os.mkdir(tile_save_path)

best tile size: 736, num pixels lost: 2648064


In [6]:
def create_tiles(im, lb, tile_size, mode_save_path, im_name):
    img = cv2.imread(im)
    label = cv2.imread(lb, cv2.IMREAD_UNCHANGED)
    h_inds = np.arange(0, img.shape[0]+1, tile_size) # horizontal tile index values ie. [0 480 960 ...]
    v_inds = np.arange(0, img.shape[1]+1, tile_size) # vertical tile index values
    tile_counter = 0
    for i in range(len(h_inds) - 1):
        for j in range(len(v_inds) - 1):
            img_tile = img[h_inds[i]:h_inds[i+1], v_inds[j]:v_inds[j+1], :]
            label_tile = label[h_inds[i]:h_inds[i+1], v_inds[j]:v_inds[j+1]]
            fn = im_name + '_0{}'.format(tile_counter)
            tile_counter += 1
            cv2.imwrite(mode_save_path + '/images/' + fn + '.jpg', img_tile)
            cv2.imwrite(mode_save_path + '/labels/'+ fn + '.tif', label_tile)

In [7]:
for mode in os.listdir(raw_data_path): # 'train', 'test'
    mode_save_path = os.path.join(tile_save_path, mode)
    if not os.path.exists(mode_save_path): # make train/, test/ folders
        os.mkdir(mode_save_path)      
        os.mkdir(mode_save_path + '/images')
        os.mkdir(mode_save_path + '/labels')
    images_path = os.path.join(raw_data_path, mode + '/images')
    labels_path = os.path.join(raw_data_path, mode + '/labels')    
    for fn in os.listdir(images_path): # for each image
        if fn == 'Thumbs.db':
            continue
        im_name = fn[:-4]
        im = os.path.join(images_path, im_name + '.jpg')
        lb = os.path.join(labels_path, im_name + '.tif')
        print('processing {}...'.format(im_name))
        create_tiles(im, lb, tile_size, mode_save_path, im_name)      

processing FTE306-20X-normal...
processing FTE306-20X-STIC...
processing FTE307-20X-normal...
processing FTE307-20X-STIC...
processing FTE301-20X-normal...
processing FTE301-20X-STIC...
processing FTE302-20X-normal...
processing FTE302-20X-STIC...
processing FTE303-20X-normal...
processing FTE303-20X-STIC...
processing FTE304-20X-normal...
processing FTE304-20X-STIC...
processing FTE305-20X-normal...
processing FTE305-20X-STIC...
